# Find TS conformers and calculate energy on the fly
This uses a RDkit as backend to generate the TS conformers and calculate the energy by Psi4.

Note: if the jupyter notebook crashes. It is highly possible that the `openbabel` and `rdkit` <br>
uses different dependencies during compiling, assuming you are using conda builds.<br>
You can either try to build your own version (solve the problem) or change the import sequence<br>
(can make the latter one malfunctioning when use some of the methods)<br>

## 1. Generate conformers
Necessary packages

In [ ]:
import sys
sys.path.append('/Users/xiaorui/Apps/ACS')

In [ ]:
import os
import multiprocessing
from copy import deepcopy
from itertools import combinations, product
from typing import List, Optional, Union

import numpy as np
import psi4
from joblib import Parallel, delayed
from rdkit import Chem

from acs.backend.rdk import RDKitConf, RDKitTS
from acs.converter.geom import (xyz_str_to_xyz_dict,
                                xyz_dict_to_xyz_str,
                                xyz_dict_to_xyz_file)
import pybel

%load_ext autoreload
%load_ext autotime
%autoreload 2
%matplotlib inline

In [ ]:
def get_separable_angle_list(samplings: Union[list, tuple],
                             from_angles: Optional[Union[list, tuple]] = None):
    """
    Get a angle list for each input dimension. For each dimension
    The input can be a int, indicating the angles will be evenly sampled;
    Or a list, indicate the angles to be sampled;
    Examples:
    [[120, 240,], 4, 0] => [np.array([120, 240,]), 
                            np.array([0, 90, 180, 270,]), 
                            np.array([0])]
    List of np.arrays are returned for the sake of further calculation
    
    Args:
        samplings (Union[list, tuple]): An array of sampling information.
                  For each element, it can be either list or int.
        from_angles (Union[list, tuple]): An array of initial angles.
                    If not set, angles will begin at zeros.
    
    Returns:
        list: A list of sampled angles sets.
    """
    from_angles = from_angles or len(samplings) * [0.]
    angle_list = []
    for ind, angles in enumerate(samplings):
        # Only provide a number
        # This is the step number of the angles
        if isinstance(angles, (int, float)):
            try:
                step = 360 // angles
            except ZeroDivisionError:
                # Does not change
                angles = from_angles[ind] + np.array([0])
            else:
                angles = from_angles[ind] + \
                         np.array([step * i for i in range(angles)])
        elif isinstance(angles, list):
            angles = from_angles[ind] + np.array(angles)
            
        # Set to 0 - 360 range
        for i in range(angles.shape[0]):
            while angles[i] < 0.:
                angles[i] += 360
            while angles[i] > 360.:
                angles[i] -= 360

        angle_list.append(angles.tolist())            
    return angle_list


def conformers_by_change_torsions(conf: 'RDKitConf',
                                  angle_mesh,
                                  bookkeep: dict,
                                  torsions=None,
                                  on_the_fly_check=False):
    """
    Generate conformers by rotating the angles of the torsions. The result will be saved into
    ``bookkeep``. A on-the-fly check can be applied, which identifies the conformers with colliding
    atoms.
    
    Args:
        conf (RDkitConf): A RDKit Conformer to be used.
        angle_mesh (iterable): An iterable contains the angle_list for conformers to be generated from.
        bookkeep (dict): A dictionary to save the coords.
        torsions (list): A list of four-atom-index lists indicating the torsional modes.
        on_the_fly_filter (bool): Whether to check colliding atoms on the fly.
    """
    if not torsions:
        torsions = conf.GetTorsionalModes()
        for ind, angles in enumerate(angle_mesh):
            conf.SetAllTorsionsDeg(angles)
            bookkeep[ind] = {'angles': angles,
                             'coords': conf.GetPositions().tolist()}
            bookkeep[ind]['colliding_atoms'] = conf.HasCollidingAtoms() \
                if on_the_fly_check == True else None

    else:
        all_torsions = conf.GetTorsionalModes()
        try:
            changing_torsions_index = [all_torsions.index(tor) for tor in torsions]
        except ValueError as e:
            # tor not in all_torsions
            raise
            
        original_angles = conf.GetAllTorsionsDeg()

        for ind, angles in enumerate(angle_mesh):
            for i, angle, tor in zip(range(len(angles)), angles, torsions):
                conf.SetTorsionDeg(tor, angle)
                original_angles[changing_torsions_index[i]] = angle
                
            bookkeep[ind] = {'angles': original_angles,
                             'coords': conf.GetPositions().tolist()}
            bookkeep[ind]['colliding_atoms'] = conf.HasCollidingAtoms() \
                    if on_the_fly_check == True else None

## 1.1 Perceive a molecule from xyz

In [ ]:
######################################
# INPUT
xyz_str="""C          0.86917        1.50073       -0.41702
C         -0.47607        2.03752        0.01854
H          0.85087        0.99684       -1.38520
H          1.16254        0.53083        0.35265
C         -1.66031        1.18407       -0.41323
H         -0.48474        2.19298        1.10199
H         -0.61511        3.02720       -0.42857
N         -1.64328       -0.19909        0.05343
H         -1.68450        1.15374       -1.50571
H         -2.58945        1.68138       -0.08673
C         -2.74796       -0.93786       -0.53898
C         -1.67887       -0.29955        1.50384
H         -2.69951       -1.98186       -0.22875
H         -3.72671       -0.52994       -0.24178
H         -2.67337       -0.90452       -1.62638
H         -1.69925       -1.35091        1.79085
H         -2.56817        0.19427        1.92693
H         -0.78293        0.13647        1.93981
O          1.61241       -0.39643        1.10878
O          2.26285       -1.25639        0.24798
C          1.32762       -2.20908       -0.22666
O          0.43662       -1.69114       -1.13896
H          1.94625       -2.95523       -0.72774
H          0.82192       -2.64082        0.64549
H         -0.24063       -1.15943       -0.66324
N          1.91323        2.44655       -0.35205
H          1.98740        2.90733        0.54373
H          2.80917        2.06603       -0.61836
"""
######################################

## 1.1 Perceive TS

### 1.1.1 Pybel (openbabel) is used to roughly perceive the molecular connectivity

In [ ]:
xyz_file = f"{len(xyz_str.splitlines())}\n\n{xyz_str}"
xyz_dict = xyz_str_to_xyz_dict(xyz_str)
pybel_mol = pybel.readstring('xyz', xyz_file)

rdkitts = RDKitTS.FromOBMol(pybel_mol.OBMol)

rdkitts.EmbedConformer()
conf = rdkitts.GetConformer()
conf.SetPositions(xyz_dict['coords'])

### 1.1.2 Set the missing bonds existing in the TS. 
You can either set manually or use distance matrix and derived covalent matrix to infer which bonds are missing

In [ ]:
######################################
# INPUT
bonds = []
threshold = 1.6
######################################
print('Current connectivity perception:')
display(rdkitts)
if not bonds:
    dist_mat = np.triu(rdkitts.GetDistanceMatrix())
    covl_mat = rdkitts.GetCovalentMatrix()

    for multiplier in np.arange(1.1, threshold, 0.1):
        atom1s, atom2s = np.where((dist_mat - multiplier * covl_mat) < 0)
        bonds = [(int(atom1s[i]), int(atom2s[i])) for i in range(len(atom1s))]
        if bonds:
            print(f"Possible transition state bonds: {bonds} (with threshold = {multiplier:.1f})")

### 1.1.3 Overwrite the RDKitTS with new bonding info

In [ ]:
rw_mol = rdkitts.ToRDMol()

for bond in bonds:
    # Use double bond to avoid to be counted as a torsional mode
    # If you want to include it, please use BondType.SINGLE
    rw_mol.AddBond(*bond, Chem.BondType.DOUBLE)
    
rdkitts = rdkitts.FromRDMol(rw_mol)
print('Modified Molecule:')
display(rdkitts)

## 1.2 Use RDKit to generate conformers

### 1.2.1 Initialize a conformer instance

In [ ]:
# Create a Conformer instance and set the initial xyz
rdkitts.EmbedConformer()
conf = rdkitts.GetConformer()
conf.SetPositions(xyz_dict['coords'])

### 1.2.2 Get the torsional mode and the original angles

In [ ]:
# You can set the correct (all) torsions, otherwise RDKit will perceive.
######################################
# INPUT
torsions = None
######################################
if not torsions:
    torsions = rdkitts.GetTorsionalModes()
    print(f'RDKit perceived torsions: {torsions}')

conf.SetTorsionalModes(torsions)
num_torsions = len(torsions)
original_angles = conf.GetAllTorsionsDeg()
print('Torsions highlighted in the molecule:')
display(rdkitts)
print(f'The original dihedral angles is: {original_angles}')

### 1.2.3 Generate conformers according to the mangle mesh

Example1: Sampling the angles `0, 120, 240` for each torsion <br>
For reference, a 7 heavy atom species with 5 rotors cost ~20 ms on Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz

In [ ]:
sampling = [3] * len(torsions)  # sampling = [[0., 120., 240.]] is equivalent

In [ ]:
angles_list = get_separable_angle_list(sampling,
                                       original_angles)
angle_mesh = product(*angles_list)

# Save to bookkeep
bookkeep = {}
conformers_by_change_torsions(conf,
                              angle_mesh,
                              bookkeep=bookkeep,
                              torsions=torsions,
                              on_the_fly_check=True)

Example2: Sampling the angles 45 x 45 points for each torsion pairs <br>
For reference, a 7 heavy atom species with 5 rotors cost 1.4 s on Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz

In [ ]:
n_point_each_torsion = 20
n_dimension = 2

In [ ]:
bookkeeps = []

for torsion_pair in combinations(torsions, n_dimension):
    # Reset the geometry
    conf.SetPositions(xyz_dict['coords'])
    # Get angles
    sampling = [n_point_each_torsion if tor in torsion_pair else 0
                for tor in torsions]
    angles_list = get_separable_angle_list(sampling,
                                           original_angles)
    angle_mesh = product(*angles_list)
    # Generate conformers
    bookkeep = {}
    conformers_by_change_torsions(conf,
                                  angle_mesh,
                                  bookkeep=bookkeep,
                                  torsions=torsions,
                                  on_the_fly_check=True)
    bookkeeps.append(bookkeep)

### [OPTIONAL] Check colliding atoms of conformers
if you run with on-the-fly check you can check the colliding atoms

In [ ]:
# check_bookkeep = [bookkeep]  # Example 1
check_bookkeep = bookkeeps # Example 2

In [ ]:
total_collide = 0
for i, bk in enumerate(check_bookkeep):
    inds = []
    for ind, value in bk.items():
        if value['colliding_atoms']:
            inds.append(ind)
    total_collide += len(inds)
    print(f"Set {i}: {len(inds)} out of {len(bk)} conformers may have colliding atoms.")
print(f"Total: {total_collide} out of {len(bk) * len(check_bookkeep)} conformers may have colliding atoms.")

### [OPTIONAL] Check volume of the TS
You can check the distribution of the molecule exvolume to see whether the molecule is apart. It is possible to see smaller volume due to folding or colliding.

In [ ]:
import seaborn as sns
import pandas as pd

# check_bookkeep = [bookkeep]  # Example 1
check_bookkeep = bookkeeps  # Example 2
random_points = 10  # each group

In [ ]:
rdkitts.EmbedConformer()
conf = rdkitts.GetConformer()
rd_mol = rdkitts.ToRDMol()

all_volumes = []
for bk in check_bookkeep:
    rnd_sample = np.random.randint(400, size=10)
    volume = np.zeros_like(rnd_sample)
    for index in range(rnd_sample.shape[0]):
        conf.SetPositions(bk[index]['coords'])
        volume[index] = Chem.AllChem.ComputeMolVolume(rd_mol)
    all_volumes.append(volume)

In [ ]:
ax = sns.violinplot(data=all_volumes)
ax.set_xlabel('group index')
ax.set_ylabel('volume')
for tick in ax.get_xticklabels():
    tick.set_rotation(90)

### [OPTIONAL] Align Conformers
You can also align molecules stored in the `bookkeep`. The RMSD of each conformer (initial conformer as reference) is also generated `rmsd_list`

In [ ]:
# to_align = bookkeep  # Example 1
to_align = bookkeeps[3]  # Example 2

In [ ]:
from rdkit import Chem

rdkitts.EmbedMultipleConfs(len(to_align))
confs = rdkitts.GetAllConformers()
torsions_values = []
for i, value in to_align.items():
    confs[i].SetPositions(value['coords'])
    torsion_value = np.array(confs[i].GetAllTorsionsDeg())
    torsions_values.append(torsion_value)
rd_mol = rdkitts.ToRDMol()

rmsd_list = []
Chem.rdMolAlign.AlignMolConformers(rd_mol, maxIters=50, RMSlist=rmsd_list)

View alignment by py3Dmol, this may take some time. {

In [ ]:
import py3Dmol

view = py3Dmol.view(width=600, height=400)
for i in range(len(bookkeep)):
    view.addModel(Chem.MolToMolBlock(rd_mol, confId=i), 'sdf')
view.setStyle({'stick':{'colorscheme':'cyanCarbon'}})
view.render()

## 2. Calculate Psi4 on the notebook

This section is only for testing instead of actual tasks.
From experience, for conformer search purpose, better to use `n_threads_each_calculation = 1` and use `n_worker` as many as possible

In [ ]:
# How many threads to use as worker
n_worker = 8  # -1 to use all threads
n_memory_each_calculation = 12000 / n_worker  # Assuming you have 
n_threads_each_calculation = 1
reference = 'uhf'
level_of_theory = 'b3lyp/def2-svp'

In [ ]:
def geom_producer(bookkeep, xyz_dict):
    for ind, conf in bookkeep.items():
        xyz_dict['coords'] = conf['coords']
        xyz_file = xyz_dict_to_xyz_file(xyz_dict)
        yield (ind, xyz_file)
        
def get_psi4_dftenergy(ind, xyz_file):
    psi4.set_memory(f'{n_memory_each_calculation} MB')
    psi4.set_options({'reference': reference})
    try:
        psi4.geometry(xyz_file)
        psi4.set_num_threads(n_threads_each_calculation)
        return (ind, psi4.energy(level_of_theory))
    except Exception as e:
        print(e)
        return (ind, 1e4)

In [ ]:
result = Parallel(n_jobs=n_worker, verbose=100) \
         (delayed(get_psi4_dftenergy)(*data) for data in geom_producer(bookkeep, xyz_dict))